In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, Input

In [2]:
# Load data from JSON file
file_path = 'C:/Users/HP/Desktop/angles_output_3.json'
data = []
with open(file_path) as f:
    for line in f:
        data.append(json.loads(line))

In [3]:
# Convert JSON data to DataFrame
df = pd.DataFrame(data)

In [4]:
# Separate features and labels
X = df.drop('Label', axis=1).values
y = df['Label'].values

In [5]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [7]:
# Reshape data for RNN
X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [8]:
# Build RNN model
rnn_model = Sequential()
rnn_model.add(Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])))
rnn_model.add(SimpleRNN(50))
rnn_model.add(Dropout(0.2))
rnn_model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [9]:
# Compile model
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train model
rnn_history = rnn_model.fit(X_train_rnn, y_train, epochs=50, batch_size=32, validation_data=(X_test_rnn, y_test))

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1850 - loss: 2.2794 - val_accuracy: 0.2698 - val_loss: 1.6376
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2846 - loss: 1.6458 - val_accuracy: 0.5344 - val_loss: 1.2963
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4278 - loss: 1.3964 - val_accuracy: 0.5344 - val_loss: 1.2411
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5267 - loss: 1.2742 - val_accuracy: 0.5556 - val_loss: 1.1993
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5456 - loss: 1.2255 - val_accuracy: 0.5820 - val_loss: 1.1234
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5626 - loss: 1.1898 - val_accuracy: 0.5767 - val_loss: 1.0399
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6050 - loss: 1.0710 - val_accuracy: 0.5450 - val_loss: 1.0094
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6435 - loss: 1.0252 - val_accuracy: 0.6614 - val_loss

In [11]:
# Save the training history
with open('rnn_training_history.json', 'w') as f:
    json.dump(rnn_history.history, f)

In [12]:
# Save RNN model
rnn_model.save('verification_rnn_model.h5')